### Day 15: Warehouse Woes

Link: https://adventofcode.com/2024/day/15

We can solve the problem by sequentially processing all commands. To determine if a robot or box can move into a position, we can create a recursive method that checks whether the current position is free for the agent (either the robot or a box) attempting to enter it. If the current position contains a box, the method is called again with the new position where the box would be moved. Then, if moving the robot and all necessary boxes is possible, we apply the changes and update the robot's position. Given a command length of 20k and a grid size of 50x50, this process should require at most 1M iterations (`number of commands x max(grid height, grid width)`).

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
from dataclasses import dataclass
from collections import deque


@dataclass
class Position:
    row: int
    column: int


@dataclass
class Robot:
    position: Position


grid: list[list[int]] = []
robot = Robot(position=Position(row=-1, column=-1))


for row, line in enumerate(lines):
    line = line.strip()

    if not line:
        break

    grid_row: list[str] = []

    for column, cell in enumerate(line):
        grid_row.append(cell)

        if cell == "@":
            robot.position.row = row
            robot.position.column = column

    grid.append(grid_row)


commands: list[str] = []


for row in range(row + 1, len(lines)):
    line = lines[row].strip()
    commands.extend(list(line))


actions = {
    "^": (-1, 0),
    "v": (1, 0),
    "<": (0, -1),
    ">": (0, 1),
}


def make_command_new_row_column(command: str, row: int, column: int) -> tuple[int, int]:
    return row + actions[command][0], column + actions[command][1]


def can_update_grid(command: str, row: int, column: int) -> bool:
    if grid[row][column] == "#":
        return False

    if grid[row][column] == ".":
        return True

    new_row, new_column = make_command_new_row_column(command, row, column)
    return can_update_grid(command, new_row, new_column)


def update_grid(command: str, robot: Robot) -> None:
    to_update = deque([(".", robot.position.row, robot.position.column)])

    while to_update:
        new_cell_value, row, column = to_update.popleft()
        old_cell_value = grid[row][column]
        grid[row][column] = new_cell_value

        if new_cell_value == "@":
            robot.position.row = row
            robot.position.column = column

        if old_cell_value not in {".", "#"}:
            new_row, new_column = make_command_new_row_column(command, row, column) 
            to_update.append((old_cell_value, new_row, new_column))


for command in commands:
    if can_update_grid(command, robot.position.row, robot.position.column):
        update_grid(command, robot)


sum_coordinates = 0


for row in range(len(grid)):
    for column, cell in enumerate(grid[row]):
        if cell == "O":
            sum_coordinates += row * 100 + column


print(sum_coordinates)